In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Bước 1: Đọc dữ liệu từ file CSV
data = pd.read_csv("du_lieu_hoa_qua.csv")

# Kiểm tra dữ liệu
print("Dữ liệu mẫu:")
print(data.head())

# Bước 2: Dataset tùy chỉnh
class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Lấy câu hỏi và câu trả lời từ cột "Câu trả lời"
        row = self.data.iloc[idx]
        input_text = "Huấn luyện: " + row["Câu trả lời"]
        output_text = row["Câu trả lời"]

        # Token hóa
        encoding = self.tokenizer(
            input_text, 
            max_length=self.max_length, 
            truncation=True, 
            padding="max_length",
            return_tensors="pt"
        )
        labels = self.tokenizer(
            output_text, 
            max_length=self.max_length, 
            truncation=True, 
            padding="max_length",
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze()
        }

# Bước 3: Chuẩn bị dữ liệu và tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

dataset = QADataset(data, tokenizer)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Bước 4: Tải mô hình GPT-2
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Bước 5: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_total_limit=2
)

# Bước 6: Huấn luyện
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

print("Bắt đầu quá trình huấn luyện...")
trainer.train()

# Bước 7: Lưu mô hình
model.save_pretrained("./gpt2_hoa_qua")
tokenizer.save_pretrained("./gpt2_hoa_qua")

print("Huấn luyện hoàn tất và mô hình đã được lưu tại ./gpt2_hoa_qua")
